# Aspect-Based Sentiment Analysis with Multi-Class SVM

This notebook implements aspect-based sentiment analysis using:
- Feature extraction: TF-IDF and Word2Vec
- Sentiment labeling: Score-based, TextBlob, VADER, and Ensemble methods
- Classification: Multi-Class SVM with various kernels
- Data splits: 65%, 70%, 75% training scenarios
- Evaluation: Confusion Matrix and ROC/AUC

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    roc_auc_score, roc_curve, auc
)
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
df = pd.read_csv('google_play_reviews_DigitalBank_sentiment_analysis.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Data exploration and visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Sentiment Distribution Across Different Methods', fontsize=16)

sentiment_columns = ['sentiment_score_based', 'sentiment_textblob', 'sentiment_vader', 'sentiment_ensemble']

for i, col in enumerate(sentiment_columns):
    ax = axes[i//2, i%2]
    df[col].value_counts().plot(kind='bar', ax=ax, title=col.replace('_', ' ').title())
    ax.set_xlabel('Sentiment')
    ax.set_ylabel('Count')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Print distribution statistics
for col in sentiment_columns:
    print(f"\n{col}:")
    print(df[col].value_counts())
    print(f"Percentage distribution:")
    print(df[col].value_counts(normalize=True) * 100)

## Feature Extraction

In [ ]:
# Prepare text data
texts = df['stemmed_text'].fillna('').astype(str)

# 1. TF-IDF Feature Extraction
print("Extracting TF-IDF features...")
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    stop_words=None  # Already preprocessed
)

tfidf_features = tfidf_vectorizer.fit_transform(texts)
print(f"TF-IDF feature shape: {tfidf_features.shape}")

# 2. Word2Vec Feature Extraction
print("\nTraining Word2Vec model...")
# Tokenize texts for Word2Vec
tokenized_texts = [simple_preprocess(text) for text in texts]

# Train Word2Vec model
w2v_model = Word2Vec(
    sentences=tokenized_texts,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    epochs=10
)

print(f"Word2Vec vocabulary size: {len(w2v_model.wv.key_to_index)}")

In [ ]:
# Function to create document vectors from Word2Vec
def get_document_vector(tokens, model, vector_size=100):
    """Create document vector by averaging word vectors"""
    vectors = []
    for token in tokens:
        if token in model.wv.key_to_index:
            vectors.append(model.wv[token])
    
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

# Create Word2Vec features
print("Creating Word2Vec document vectors...")
w2v_features = np.array([get_document_vector(tokens, w2v_model) for tokens in tokenized_texts])
print(f"Word2Vec feature shape: {w2v_features.shape}")

# Convert TF-IDF to dense array for consistency
tfidf_features_dense = tfidf_features.toarray()
print(f"TF-IDF dense feature shape: {tfidf_features_dense.shape}")

## Multi-Class SVM Classification

In [ ]:
# Prepare target variables
sentiment_methods = ['sentiment_score_based', 'sentiment_textblob', 'sentiment_vader', 'sentiment_ensemble']
feature_types = ['TF-IDF', 'Word2Vec']
kernels = ['linear', 'rbf', 'poly', 'sigmoid']
train_sizes = [0.65, 0.70, 0.75]

# Encode labels
label_encoders = {}
encoded_labels = {}

for method in sentiment_methods:
    le = LabelEncoder()
    encoded_labels[method] = le.fit_transform(df[method])
    label_encoders[method] = le
    print(f"{method} classes: {le.classes_}")

print("\nLabel encoding completed!")

In [ ]:
# Function to evaluate SVM model
def evaluate_svm_model(X_train, X_test, y_train, y_test, kernel, method_name, feature_name, train_size):
    """Train and evaluate SVM model"""
    
    # Train SVM
    svm_model = SVC(kernel=kernel, probability=True, random_state=42)
    svm_model.fit(X_train, y_train)
    
    # Predictions
    y_pred = svm_model.predict(X_test)
    y_pred_proba = svm_model.predict_proba(X_test)
    
    # Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Classification report
    class_report = classification_report(y_test, y_pred, output_dict=True)
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # ROC AUC (for multiclass)
    try:
        if len(np.unique(y_test)) > 2:
            # Multiclass ROC AUC
            roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='weighted')
        else:
            # Binary ROC AUC
            roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1])
    except:
        roc_auc = None
    
    return {
        'method': method_name,
        'feature': feature_name,
        'kernel': kernel,
        'train_size': train_size,
        'accuracy': accuracy,
        'roc_auc': roc_auc,
        'classification_report': class_report,
        'confusion_matrix': cm,
        'model': svm_model
    }

print("Evaluation function defined!")

In [ ]:
# Run comprehensive evaluation
results = []
feature_data = {
    'TF-IDF': tfidf_features_dense,
    'Word2Vec': w2v_features
}

print("Starting comprehensive SVM evaluation...")
print("This may take several minutes...\n")

total_experiments = len(sentiment_methods) * len(feature_types) * len(kernels) * len(train_sizes)
current_experiment = 0

for method in sentiment_methods:
    y = encoded_labels[method]
    
    for feature_name, X in feature_data.items():
        for train_size in train_sizes:
            test_size = 1 - train_size
            
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, train_size=train_size, random_state=42, stratify=y
            )
            
            for kernel in kernels:
                current_experiment += 1
                print(f"Experiment {current_experiment}/{total_experiments}: {method} | {feature_name} | {kernel} | Train: {train_size*100:.0f}%")
                
                try:
                    result = evaluate_svm_model(
                        X_train, X_test, y_train, y_test, 
                        kernel, method, feature_name, train_size
                    )
                    results.append(result)
                except Exception as e:
                    print(f"Error in experiment: {e}")
                    continue

print(f"\nCompleted {len(results)} successful experiments!")

## Results Analysis and Visualization

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame([
    {
        'Method': r['method'],
        'Feature': r['feature'],
        'Kernel': r['kernel'],
        'Train_Size': r['train_size'],
        'Accuracy': r['accuracy'],
        'ROC_AUC': r['roc_auc']
    }
    for r in results
])

print("Results Summary:")
print(results_df.groupby(['Method', 'Feature', 'Kernel']).agg({
    'Accuracy': ['mean', 'std'],
    'ROC_AUC': ['mean', 'std']
}).round(4))

In [ ]:
# Visualization: Accuracy comparison
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
fig.suptitle('SVM Performance Comparison Across Different Configurations', fontsize=16)

# 1. Accuracy by Method and Feature
ax1 = axes[0, 0]
sns.boxplot(data=results_df, x='Method', y='Accuracy', hue='Feature', ax=ax1)
ax1.set_title('Accuracy by Sentiment Method and Feature Type')
ax1.tick_params(axis='x', rotation=45)

# 2. Accuracy by Kernel
ax2 = axes[0, 1]
sns.boxplot(data=results_df, x='Kernel', y='Accuracy', ax=ax2)
ax2.set_title('Accuracy by SVM Kernel')

# 3. Accuracy by Training Size
ax3 = axes[1, 0]
sns.boxplot(data=results_df, x='Train_Size', y='Accuracy', ax=ax3)
ax3.set_title('Accuracy by Training Size')

# 4. ROC AUC by Method and Feature
ax4 = axes[1, 1]
results_df_clean = results_df.dropna(subset=['ROC_AUC'])
sns.boxplot(data=results_df_clean, x='Method', y='ROC_AUC', hue='Feature', ax=ax4)
ax4.set_title('ROC AUC by Sentiment Method and Feature Type')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Find best performing models
print("Top 10 Best Performing Models by Accuracy:")
top_accuracy = results_df.nlargest(10, 'Accuracy')
print(top_accuracy[['Method', 'Feature', 'Kernel', 'Train_Size', 'Accuracy', 'ROC_AUC']].to_string(index=False))

print("\n" + "="*80)
print("Top 10 Best Performing Models by ROC AUC:")
top_roc = results_df.dropna(subset=['ROC_AUC']).nlargest(10, 'ROC_AUC')
print(top_roc[['Method', 'Feature', 'Kernel', 'Train_Size', 'Accuracy', 'ROC_AUC']].to_string(index=False))

In [ ]:
# Detailed analysis of best model
best_model_idx = results_df['Accuracy'].idxmax()
best_result = results[best_model_idx]

print(f"Best Model Configuration:")
print(f"Method: {best_result['method']}")
print(f"Feature: {best_result['feature']}")
print(f"Kernel: {best_result['kernel']}")
print(f"Training Size: {best_result['train_size']*100:.0f}%")
print(f"Accuracy: {best_result['accuracy']:.4f}")
print(f"ROC AUC: {best_result['roc_auc']:.4f}")

# Get actual test data for detailed report
method = best_result['method']
feature_name = best_result['feature']
train_size = best_result['train_size']

y = encoded_labels[method]
X = feature_data[feature_name]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_size, random_state=42, stratify=y
)

# Train best model
best_svm = SVC(kernel=best_result['kernel'], probability=True, random_state=42)
best_svm.fit(X_train, y_train)
y_pred = best_svm.predict(X_test)

# Classification report for best model
print("\nDetailed Classification Report:")
class_names = label_encoders[method].classes_
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
# Confusion Matrix Visualization for Best Models
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Confusion Matrices for Best Models by Sentiment Method', fontsize=16)

# Get best model for each sentiment method
best_models_by_method = results_df.loc[results_df.groupby('Method')['Accuracy'].idxmax()]

for i, (_, row) in enumerate(best_models_by_method.iterrows()):
    if i >= 4:  # Only show first 4
        break
        
    ax = axes[i//2, i%2]
    
    # Find corresponding result
    result = next(r for r in results if (
        r['method'] == row['Method'] and 
        r['feature'] == row['Feature'] and 
        r['kernel'] == row['Kernel'] and 
        r['train_size'] == row['Train_Size']
    ))
    
    cm = result['confusion_matrix']
    class_names = label_encoders[result['method']].classes_
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=class_names, yticklabels=class_names)
    ax.set_title(f"{row['Method']}\n{row['Feature']} | {row['Kernel']} | Acc: {row['Accuracy']:.3f}")
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')

plt.tight_layout()
plt.show()

## Summary and Recommendations

In [ ]:
# Generate comprehensive summary
print("=" * 80)
print("ASPECT-BASED SENTIMENT ANALYSIS - COMPREHENSIVE SUMMARY")
print("=" * 80)

print(f"\nDataset Information:")
print(f"- Total samples: {len(df)}")
print(f"- Features extracted: TF-IDF ({tfidf_features.shape[1]} features), Word2Vec (100 dimensions)")
print(f"- Sentiment methods compared: {len(sentiment_methods)}")
print(f"- SVM kernels tested: {kernels}")
print(f"- Training sizes tested: {[f'{size*100:.0f}%' for size in train_sizes]}")
print(f"- Total experiments conducted: {len(results)}")

print(f"\nOverall Performance Summary:")
print(f"- Best accuracy achieved: {results_df['Accuracy'].max():.4f}")
print(f"- Average accuracy across all experiments: {results_df['Accuracy'].mean():.4f} ± {results_df['Accuracy'].std():.4f}")
if not results_df['ROC_AUC'].isna().all():
    print(f"- Best ROC AUC achieved: {results_df['ROC_AUC'].max():.4f}")
    print(f"- Average ROC AUC: {results_df['ROC_AUC'].mean():.4f} ± {results_df['ROC_AUC'].std():.4f}")

print(f"\nBest Performing Configuration:")
best_config = results_df.loc[results_df['Accuracy'].idxmax()]
print(f"- Sentiment Method: {best_config['Method']}")
print(f"- Feature Type: {best_config['Feature']}")
print(f"- SVM Kernel: {best_config['Kernel']}")
print(f"- Training Size: {best_config['Train_Size']*100:.0f}%")
print(f"- Accuracy: {best_config['Accuracy']:.4f}")
if pd.notna(best_config['ROC_AUC']):
    print(f"- ROC AUC: {best_config['ROC_AUC']:.4f}")

print(f"\nKey Insights:")
# Feature type performance
feature_performance = results_df.groupby('Feature')['Accuracy'].mean()
best_feature = feature_performance.idxmax()
print(f"- Best feature type: {best_feature} (avg accuracy: {feature_performance[best_feature]:.4f})")

# Kernel performance
kernel_performance = results_df.groupby('Kernel')['Accuracy'].mean()
best_kernel = kernel_performance.idxmax()
print(f"- Best SVM kernel: {best_kernel} (avg accuracy: {kernel_performance[best_kernel]:.4f})")

# Training size impact
size_performance = results_df.groupby('Train_Size')['Accuracy'].mean()
best_size = size_performance.idxmax()
print(f"- Optimal training size: {best_size*100:.0f}% (avg accuracy: {size_performance[best_size]:.4f})")

# Sentiment method performance
method_performance = results_df.groupby('Method')['Accuracy'].mean()
best_method = method_performance.idxmax()
print(f"- Best sentiment method: {best_method} (avg accuracy: {method_performance[best_method]:.4f})")

print(f"\nRecommendations:")
print(f"1. Use {best_feature} features for better performance")
print(f"2. {best_kernel} kernel shows best results for this dataset")
print(f"3. {best_size*100:.0f}% training split provides optimal balance")
print(f"4. {best_method} sentiment labeling method is most suitable")
print(f"5. Consider ensemble methods for production deployment")

print("\n" + "=" * 80)
print("Analysis completed successfully!")
print("=" * 80)